I want to have 

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# List only files in the current directory
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f)

Bandi.jpg
CopenhagenOpen2025tab.txt
EUDC2024tab.csv.xlsx
EUDC2024tab.txt
EUDC2024_rounds_done.xlsx
EUDC2024_speakertab_done.xlsx
EUDC2024_teamtab_done.xlsx
FileRenamer.ipynb
README.md
Tab2Database.ipynb


# Start: Goal
##### Debate Project
The goal of this project is to use programming to enhance the level of competitive academic debating, mostly by doing advanced data-analysis and further techniques.

##### Tabdata
A huge chapter in this is the creation of a comprehensive debate database, that includes all elements of debating that can be used to measure, evaluate, and increase performance. At this junction of the project the **TabData** project tries to set up the database of debates and some follow-up metrics, such as "elo-score". 

##### Tabdata useage
In ideal case this can be used for predicting winners of debates, predicting and analyzing biases and outliers with motions, judges, speaker-scores of certain judges, hardships with motion-types, correlations between outcomes in debate tournaments, or just providing an overview about the long-term trends within debating by having a comprehensive data through a long cross-section of time. 

# About the data
Our data arrives in a csv. <span style="color:red">**At this test programme, it is manually downloaded**</span>  but for future tournament databases, it will be downloaded by scraping with Selenium.

The data consists the outcomes of debate tournaments. This has 3 main sections. 

#### Team tab ####
The teamtab consists the teams that participated at the tournament, and the ranking between the teams at the end of the tournament's preliminary rounds. As a rule of thumb, tournaments run for 5 rounds, but major tournaments such as European or World Championships have 9 preliminary rounds. The top teams go through to the elimination rounds, to reach, and eventually win the finals. Teams that do not reach the playoffs, are ranked by their respective positions in the tab. 

For example, if there are 40 teams in a tournament, the top 8 will go to the semi-finals, with a preset seeding in British Parliamentary debates where in Semi-Final-1 the 1st, 4th, 5th, and 8th seeded teams debate, and in Semi-Final-2 the 2nd, 3rd, 6th, 7th seeded teams debate. Each semi-final will have 2 teams that advance to the finals. So, despite teams do not carry the resulst to the outrounds, they have an impact on each team's result. Teams 9-40 are ranked based on their positions within the tab. 

#### Speaker tab ####
In the same csv, following the teamtab, comes the speakertab. While the teamtab is ranking teams, the speakertab ranks the individual speakers within a debate tournaments. Because in British Parliamentary debating the speaker score of the individual speakers within a debate is summed, and teams are ranked in the order of the sum of these scores, teamtab and speakertab strongly correlates, but they don't have to be the same. Furthermore, it provides further ways to analyse data for us. Both the teamtab and the spakertab is with their own headers within the data, therefore slicing up the csv to teamtab and speakertab is not particularly complex.

#### Debate results ####
The third, and largest part of the csv consists each team's result within a given debate, and also the role they had (f.e. Opening Government) and the adjucating judges too. Each row consists the data for each individual team's separate round. 

Each round was obtained separately and attached to the csv, so every round has its own headers, that is needed, because the data does not consist the roundnumbers themselves.

The data looks something like this: 
| team              | result | side    | adjudicators  |
| :---------------- | :------: | ----: | --------------|
| Team1        |  1st   | Opening Government | Judge1,Judge2 |  
| Team2           |  1st   | Opening Government | Judge3 |
| Team3    |  2nd   | Closing Opposition |  Judge4,Judge6 |
| Team4 |  4th   | Opening Opposition | Judge3 |

From the data we can see that Team 2 and Team 4 debated in the same debate at the given round, because they had the same adjudicators. (Judge 3). *Team2* finished 2nd, and *Team4* finished 4th. This itself is not new information, because we could retrieve this information from the teamtab. However, the teamtab provides no information on the opponents, and the speaker scores the team have received. 


#### The task #### 
The teamtab, the speakertab, and the debate_results dataframes are connected by 1 common column: the team's names. Each tournament, each team has its unique name. However, it is possible for John Blue to run in the team "Blue Brothers" in 1 tournament, with his sibling Adam Blue, and run with a different teamname at a following tournament, with a different teammate, or the very same. Therefore, the hardest part once the database is created is to clear out the database from people with similar names. Fortunately, most of the times debaters have unique names. As a matter of fact, the teamtab is not even needed for the creation of the "debate-level" dataframe. 

So, for each row of debate results, we are first going to retrieve the speaker scores and the speakers for the team from the speakertab. Then we will group the 4 debating teams together to one row. And finally, we will separate each adjudicator to a different row. This will give us 1 row of data in our phenomenal database.

## Start: Read in data ## 

In [4]:
comp_name = 'Copenhagen2025'
df = pd.read_csv('CopenhagenOpen2025tab.txt', delimiter='\t')
print(len(df))
df

586


,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
0,1,Oxford A,NaN,1st,1st,1st,2nd,1st,14,824,4,1
1,2,LSE stands for Library Sex Enjoyers (a,NaN,1st,1st,1st,1st,3rd,13,817,4,0
2,3,Cambridge A,NaN,1st,1st,2nd,3rd,1st,12,824,3,1
3,4,Ljubljana Hej Slovani,ESL,1st,3rd,1st,1st,2nd,12,812,3,1
4,5,Oxford B,NaN,3rd,1st,1st,2nd,1st,12,784,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
581,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
582,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
583,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df[200:300]

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
200,129,Speaker 1,NaN,Swing 1,72,75,75,—,—,222,74.00,1.41
201,130,Speaker 2,NaN,Swing 1,72,75,74,—,—,221,73.67,1.25
202,131,Akoako,"Novice, ESL",Sorbonne The economist,69,78,72,—,—,219,73.00,3.74
203,132,Raphael Kollegger-Steger,"Novice, ESL",Sorbonne The economist,69,76,72,—,—,217,72.33,2.87
204,133,Shree Niveditha Jayakumar,"Novice, ESL",Sorbonne The pookies,70,70,—,71,—,211,70.33,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...
295,Sorbonne Tvgirlies,2nd,Opening Government,View Ballot,"Max PapaioannidesⒸ , Toni TeschkeⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,Stockholm C,3rd,Opening Opposition,View Ballot,"Max PapaioannidesⒸ , Toni TeschkeⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,WSDC Denmark Aatish &amp; Tanish,1st,Closing Government,View Ballot,"Max PapaioannidesⒸ , Toni TeschkeⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,Berlin Osclear and Belaevable,4th,Closing Opposition,View Ballot,"Max PapaioannidesⒸ , Toni TeschkeⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## I. Teamtab ##
First, we need to create the teamtab

In [7]:
# Find the index of the row with the known value
stop_index = df[df['Rk'] == 'Rk'].index[0]

In [8]:
stop_index

71

In [9]:
teamtab = df.iloc[:stop_index]
teamtab

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
0,1,Oxford A,NaN,1st,1st,1st,2nd,1st,14,824,4,1
1,2,LSE stands for Library Sex Enjoyers (a,NaN,1st,1st,1st,1st,3rd,13,817,4,0
2,3,Cambridge A,NaN,1st,1st,2nd,3rd,1st,12,824,3,1
3,4,Ljubljana Hej Slovani,ESL,1st,3rd,1st,1st,2nd,12,812,3,1
4,5,Oxford B,NaN,3rd,1st,1st,2nd,1st,12,784,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
66,67,Sorbonne The economist,ESL,4th,1st,4th,—,—,3,436,1,0
67,68,Tartu Kuulilennuteetunneliluuk,ESL,4th,4th,4th,—,—,0,408,0,0
68,69,Swing 2,NaN,—,—,—,—,4th,0,142,0,0
69,70=,Swing 3,NaN,—,—,—,—,—,—,—,0,0


In [10]:
#Remove useless data
target_column = 'Spk'
column_index = teamtab.columns.get_loc(target_column)

# Slice the DataFrame to keep columns up to and including the target column
teamtab = teamtab.iloc[:, :column_index + 1]
teamtab


,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk
0,1,Oxford A,NaN,1st,1st,1st,2nd,1st,14,824
1,2,LSE stands for Library Sex Enjoyers (a,NaN,1st,1st,1st,1st,3rd,13,817
2,3,Cambridge A,NaN,1st,1st,2nd,3rd,1st,12,824
3,4,Ljubljana Hej Slovani,ESL,1st,3rd,1st,1st,2nd,12,812
4,5,Oxford B,NaN,3rd,1st,1st,2nd,1st,12,784
...,...,...,...,...,...,...,...,...,...,...
66,67,Sorbonne The economist,ESL,4th,1st,4th,—,—,3,436
67,68,Tartu Kuulilennuteetunneliluuk,ESL,4th,4th,4th,—,—,0,408
68,69,Swing 2,NaN,—,—,—,—,4th,0,142
69,70=,Swing 3,NaN,—,—,—,—,—,—,—


In [11]:
df2 = df.drop(teamtab.index)
df2.reset_index(drop=True, inplace=True)
df2

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
0,Rk,name,category,team,R1,R2,R3,R4,R5,Total,Avg,Stdev
1,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415,83.00,0.63
2,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412,82.40,2.15
3,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412,82.40,2.24
4,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410,82.00,3.10
...,...,...,...,...,...,...,...,...,...,...,...,...
510,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## II. The speaker tab

In [12]:
stop_index2 = df2[df2['Rk'] == 'team'].index[0]
speakertab = df2.iloc[:stop_index2]

#remove top row & reset index:
speakertab = speakertab.iloc[1:].reset_index(drop=True)
speakertab

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
0,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415,83.00,0.63
1,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412,82.40,2.15
2,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412,82.40,2.24
3,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410,82.00,3.10
4,4=,David Safro,ESL,Ljubljana Hej Slovani,83,77,85,84,81,410,82.00,2.83
...,...,...,...,...,...,...,...,...,...,...,...,...
136,137=,Speaker 1,NaN,Swing 2,—,—,—,—,71,71,71.00,0.00
137,137=,Speaker 2,NaN,Swing 2,—,—,—,—,71,71,71.00,0.00
138,139=,Speaker 1,NaN,Swing 3,—,—,—,—,—,—,—,—
139,139=,Speaker 2,NaN,Swing 3,—,—,—,—,—,—,—,—


In [13]:
#Remove useless data
target_column = 'Spk'
column_index = speakertab.columns.get_loc(target_column)

# Slice the DataFrame to keep columns up to and including the target column
speakertab = speakertab.iloc[:, :column_index + 1]
speakertab

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk
0,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415
1,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412
2,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412
3,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410
4,4=,David Safro,ESL,Ljubljana Hej Slovani,83,77,85,84,81,410
...,...,...,...,...,...,...,...,...,...,...
136,137=,Speaker 1,NaN,Swing 2,—,—,—,—,71,71
137,137=,Speaker 2,NaN,Swing 2,—,—,—,—,71,71
138,139=,Speaker 1,NaN,Swing 3,—,—,—,—,—,—
139,139=,Speaker 2,NaN,Swing 3,—,—,—,—,—,—


In [14]:
speakertab = speakertab.rename(columns={'Rk': 'SpeakRank','team':'speaker','R1':'team'})
speakertab

,SpeakRank,speaker,categories,team,R2,R3,R4,R5,Pts,Spk
0,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415
1,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412
2,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412
3,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410
4,4=,David Safro,ESL,Ljubljana Hej Slovani,83,77,85,84,81,410
...,...,...,...,...,...,...,...,...,...,...
136,137=,Speaker 1,NaN,Swing 2,—,—,—,—,71,71
137,137=,Speaker 2,NaN,Swing 2,—,—,—,—,71,71
138,139=,Speaker 1,NaN,Swing 3,—,—,—,—,—,—
139,139=,Speaker 2,NaN,Swing 3,—,—,—,—,—,—


In [15]:
# Get the first 4 and last column names
first_columns = speakertab.columns[:4].tolist()
last_column = speakertab.columns[-1]

# Get the middle columns that need renaming
#These will be the round results
middle_columns = speakertab.columns[4:-1]

# Create new names for the middle columns (e.g., 'NewCol1', 'NewCol2', ...)
new_middle_columns = [f'R{i}' for i in range(1, len(middle_columns) + 1)]

# Combine the first columns, renamed middle columns, and last column
new_columns = first_columns + new_middle_columns + [last_column]

# Apply the new column names to the DataFrame
speakertab.columns = new_columns

speakertab

,SpeakRank,speaker,categories,team,R1,R2,R3,R4,R5,Spk
0,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415
1,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412
2,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412
3,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410
4,4=,David Safro,ESL,Ljubljana Hej Slovani,83,77,85,84,81,410
...,...,...,...,...,...,...,...,...,...,...
136,137=,Speaker 1,NaN,Swing 2,—,—,—,—,71,71
137,137=,Speaker 2,NaN,Swing 2,—,—,—,—,71,71
138,139=,Speaker 1,NaN,Swing 3,—,—,—,—,—,—
139,139=,Speaker 2,NaN,Swing 3,—,—,—,—,—,—


## III. Connecting the speakertab & Debate Results

In [16]:
df3 = df2.drop(speakertab.index)
df3 = df3.iloc[1:].reset_index(drop=True)
df3.reset_index(drop=True, inplace=True)
df3

,Rk,team,categories,R1,R2,R3,R4,R5,Pts,Spk,1sts,2nds
0,team,result,side,ballot,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,München tja,2nd,Opening Government,View Ballot,"Annie SchwartzⒸ , Dora GnadigⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tartu Kuulilennuteetunneliluuk,4th,Opening Opposition,View Ballot,"Annie SchwartzⒸ , Dora GnadigⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tartu A,1st,Closing Government,View Ballot,"Annie SchwartzⒸ , Dora GnadigⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Oxford B,3rd,Closing Opposition,View Ballot,"Annie SchwartzⒸ , Dora GnadigⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
368,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df3.loc[df3['R1'] == 'View Ballot', 'R1'] = df3['R2']
df3 = df3.iloc[:,:4]
df3["Round"] = 0
df3

C:\Users\andra\AppData\Local\Temp\ipykernel_32020\1722710906.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["Round"] = 0


,Rk,team,categories,R1,Round
0,team,result,side,ballot,0
1,München tja,2nd,Opening Government,"Annie SchwartzⒸ , Dora GnadigⓉ",0
2,Tartu Kuulilennuteetunneliluuk,4th,Opening Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",0
3,Tartu A,1st,Closing Government,"Annie SchwartzⒸ , Dora GnadigⓉ",0
4,Oxford B,3rd,Closing Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",0
...,...,...,...,...,...
368,team,result,side,adjudicators,0
369,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",0
370,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",0
371,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",0


In [18]:
round_counter = 0
row_numbers = df3.index[df3["Rk"] == "team"].tolist()

#Trick "dumbass": In order for the next for loop to iterate through, I append a 0 to the end of my list. 
#This is needed, so when the loop is at the last round of the we don't end up with referring to a roundnumber, that would be 
#Larger than the last element in the list. 
#What I mean: check a few lines down
row_numbers.append(0)
print(row_numbers)


df4 = df3
for i in df4.index:
    if row_numbers[round_counter] == i:
        print(i)
        print(round_counter)
        
        #Trick "dumbass" --> in the last round if I did not append a 0, row_umbers[round_counter] would be
        #larger than the last element's position in the row_numbers list
        #But things just work perfectly this way, like a shitty spagetthi
        round_counter += 1
    df4.loc[i, "Round"] = round_counter  # Example logic
row_numbers = row_numbers[:-1]
df4

[0, 69, 138, 207, 272, 337, 354, 363, 368, 0]
0
0
69
1
138
2
207
3
272
4
337
5
354
6
363
7
368
8


,Rk,team,categories,R1,Round
0,team,result,side,ballot,1
1,München tja,2nd,Opening Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1
2,Tartu Kuulilennuteetunneliluuk,4th,Opening Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1
3,Tartu A,1st,Closing Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1
4,Oxford B,3rd,Closing Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1
...,...,...,...,...,...
368,team,result,side,adjudicators,9
369,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9
370,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9
371,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9


In [19]:
df4.head(20)

,Rk,team,categories,R1,Round
0,team,result,side,ballot,1
1,München tja,2nd,Opening Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1
2,Tartu Kuulilennuteetunneliluuk,4th,Opening Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1
3,Tartu A,1st,Closing Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1
4,Oxford B,3rd,Closing Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1
5,Debate partners with drawbacks,3rd,Opening Government,"Lucy MurphyⒸ , Max Papaioannides",1
6,Vilnius One and a half Greek men,2nd,Opening Opposition,"Lucy MurphyⒸ , Max Papaioannides",1
7,Berlin Osclear and Belaevable,4th,Closing Government,"Lucy MurphyⒸ , Max Papaioannides",1
8,Ljubljana Hej Slovani,1st,Closing Opposition,"Lucy MurphyⒸ , Max Papaioannides",1
9,München Munich🍺,3rd,Opening Government,Martha Johanne SchouⒸ,1


In [20]:
df4.tail(20)

,Rk,team,categories,R1,Round
353,Oxford B,eliminated,Closing Opposition,"Maša PopeskovićⒸ , Ana Coman , Jonathan Krapp",6
354,team,result,side,adjudicators,7
355,Oxford A,advancing,Opening Government,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",7
356,Ljubljana Hej Slovani,advancing,Opening Opposition,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",7
357,Tallinn unseen university a,eliminated,Closing Government,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",7
358,Newydd Aburestvičevci A,eliminated,Closing Opposition,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",7
359,WSDC Denmark Coach Laura is abusive please help,eliminated,Opening Government,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",7
360,LSE stands for Library Sex Enjoyers (a,advancing,Opening Opposition,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",7
361,Cambridge A,advancing,Closing Government,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",7
362,Belgrade BK,eliminated,Closing Opposition,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",7


In [21]:
df7 = df4
df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
for i in df7.index:
    #print(df7.Rk[i])
    loop_speaker1 = 'iron/swing'
    loop_speaker2 = 'iron/swing'
    loop_speaker1_score = 0
    loop_speaker2_score = 0
    loop_round = df7.Round[i]
    for j in speakertab.index:
        if df7.Rk[i] == speakertab.team[j]:
            if loop_speaker1 == 'iron/swing':
                loop_speaker1 = speakertab.speaker[j]
                if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                    loop_speaker1_score = speakertab.iat[j,3+loop_round]
                else:
                    loop_speaker1_score = 'outround'
            else:
                loop_speaker2 = speakertab.speaker[j]
                if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                    loop_speaker2_score = speakertab.iat[j,3+loop_round]
                else:
                    loop_speaker2_score = 'outround'
    df7.Speaker1[i] = loop_speaker1
    df7.Speaker2[i] = loop_speaker2
    df7.Speaker1_score[i] = loop_speaker1_score
    df7.Speaker2_score[i] = loop_speaker2_score
df7

C:\Users\andra\AppData\Local\Temp\ipykernel_32020\973372616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
C:\Users\andra\AppData\Local\Temp\ipykernel_32020\973372616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
C:\Users\andra\AppData\Local\Temp\ipykernel_32020\973372

,Rk,team,categories,R1,Round,Speaker1,Speaker2,Speaker1_score,Speaker2_score,Comp_name
0,team,result,side,ballot,1,iron/swing,iron/swing,0,0,Copenhagen2025
1,München tja,2nd,Opening Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1,Philipp Steen,Adora Sabau,79,77,Copenhagen2025
2,Tartu Kuulilennuteetunneliluuk,4th,Opening Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1,Hans Marten,Kaur Saarits,74,72,Copenhagen2025
3,Tartu A,1st,Closing Government,"Annie SchwartzⒸ , Dora GnadigⓉ",1,Kristjan Suuder,Tanel Mütt,80,79,Copenhagen2025
4,Oxford B,3rd,Closing Opposition,"Annie SchwartzⒸ , Dora GnadigⓉ",1,Nicole Wong,Alex Chan,77,76,Copenhagen2025
...,...,...,...,...,...,...,...,...,...,...
368,team,result,side,adjudicators,9,iron/swing,iron/swing,0,0,Copenhagen2025
369,München tja,advancing,Opening Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9,Philipp Steen,Adora Sabau,outround,outround,Copenhagen2025
370,WSDC Denmark Peter &amp; Haniya,eliminated,Opening Opposition,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9,Peter Nyvold Bojsen-Møller,Haniya Yusuf*,outround,outround,Copenhagen2025
371,INSEAD FT#3,eliminated,Closing Government,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",9,Zhangyang Shen,Avinash Pulugurtha,outround,outround,Copenhagen2025


## IV. Grouping the teambased result data up to debate based result data

In [22]:
df8 = df7
df8

grouped = df8.groupby(['Round', 'R1']).agg(list).reset_index()

# Flatten the grouped DataFrame by splitting the lists into separate columns
# Here, we use the `apply(pd.Series)` to convert each list into a separate column
grouped_expanded = pd.concat([grouped[['Round', 'R1']], 
                              grouped['Rk'].apply(pd.Series), 
                              grouped['categories'].apply(pd.Series), 
                              grouped['team'].apply(pd.Series),
                              grouped['Speaker1'].apply(pd.Series),
                              grouped['Speaker2'].apply(pd.Series),
                              grouped['Speaker1_score'].apply(pd.Series),
                              grouped['Speaker2_score'].apply(pd.Series)], axis=1)

# Rename the columns to match the desired output


In [23]:
grouped_expanded.columns = ['Round', 'Panel', 
                            'Team1', 'Team2', 'Team3', 'Team4',
                            'Team1 position', 'Team2 position', 'Team3 position', 'Team4 position',
                            'Team1 result', 'Team2 result', 'Team3 result', 'Team4 result',
                            'T1_S1 name', 'T2_S1 name', 'T3_S1 name', 'T4_S1 name',
                            'T1_S2 name', 'T2_S2 name', 'T3_S2 name', 'T4_S2 name',
                            'T1_S1 score', 'T2_S1 score', 'T3_S1 score', 'T4_S1 score',
                            'T1_S2 score', 'T2_S2 score', 'T3_S2 score', 'T4_S2 score']

In [24]:
grouped_expanded

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kshitij Aggarwal,Stefan-Christian Milicescu,75,82,73,84,74,81,73,83
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Ferdinand Ermel,Uzair Tajuddin,80,76,73,81,78,70,72,81
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Tanel Mütt,Alex Chan,79,74,80,77,77,72,79,76
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Enes Taha Erdogan,Akash Subramanian,76,78,76,75,75,77,76,73
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kristian Lindved Jensen,Dimi Kostadinov,69,74,76,78,69,74,76,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
96,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Sophie Hannigan,Maj Hrovatin,outround,outround,outround,outround,outround,outround,outround,outround
97,8,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
98,9,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",München tja,WSDC Denmark Peter &amp; Haniya,INSEAD FT#3,Leiden Peter Singer enjoyers,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Avinash Pulugurtha,Daan Koning,outround,outround,outround,outround,outround,outround,outround,outround


In [25]:
df9 = grouped_expanded
df9

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kshitij Aggarwal,Stefan-Christian Milicescu,75,82,73,84,74,81,73,83
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Ferdinand Ermel,Uzair Tajuddin,80,76,73,81,78,70,72,81
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Tanel Mütt,Alex Chan,79,74,80,77,77,72,79,76
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Enes Taha Erdogan,Akash Subramanian,76,78,76,75,75,77,76,73
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kristian Lindved Jensen,Dimi Kostadinov,69,74,76,78,69,74,76,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
96,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Sophie Hannigan,Maj Hrovatin,outround,outround,outround,outround,outround,outround,outround,outround
97,8,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
98,9,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",München tja,WSDC Denmark Peter &amp; Haniya,INSEAD FT#3,Leiden Peter Singer enjoyers,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Avinash Pulugurtha,Daan Koning,outround,outround,outround,outround,outround,outround,outround,outround


In [26]:
df9 = df9.drop(df9[df9['Team1'] == 'team'].index)
df9 = df9.reset_index()
df9 = df9.drop('index', axis=1)
df9

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kshitij Aggarwal,Stefan-Christian Milicescu,75,82,73,84,74,81,73,83
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Ferdinand Ermel,Uzair Tajuddin,80,76,73,81,78,70,72,81
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Tanel Mütt,Alex Chan,79,74,80,77,77,72,79,76
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Enes Taha Erdogan,Akash Subramanian,76,78,76,75,75,77,76,73
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Kristian Lindved Jensen,Dimi Kostadinov,69,74,76,78,69,74,76,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,6,"Velina AndonovaⒸ , Leo Marinopoulos , Marília ...",Cambridge A,Belgrade BK,EUDU Wurdaan in Copenhagen? The west ha,Tartu A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Tom Milling,Tanel Mütt,outround,outround,outround,outround,outround,outround,outround,outround
87,7,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",Oxford A,Ljubljana Hej Slovani,Tallinn unseen university a,Newydd Aburestvičevci A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Ellen Roper,Zach Godley-McAvoy,outround,outround,outround,outround,outround,outround,outround,outround
88,7,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",WSDC Denmark Coach Laura is abusive please help,LSE stands for Library Sex Enjoyers (a,Cambridge A,Belgrade BK,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Sophie Hannigan,Strahinja Batanjski,outround,outround,outround,outround,outround,outround,outround,outround
89,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Sophie Hannigan,Maj Hrovatin,outround,outround,outround,outround,outround,outround,outround,outround


In [27]:
df5 = df4
df5

grouped = df5.groupby(['Round', 'R1']).agg(list).reset_index()

# Flatten the grouped DataFrame by splitting the lists into separate columns
# Here, we use the `apply(pd.Series)` to convert each list into a separate column
grouped_expanded = pd.concat([grouped[['Round', 'R1']], 
                              grouped['Rk'].apply(pd.Series), 
                              grouped['categories'].apply(pd.Series), 
                              grouped['team'].apply(pd.Series)], axis=1)

# Rename the columns to match the desired output
grouped_expanded.columns = ['Round', 'Panel', 
                            'Team1', 'Team2', 'Team3', 'Team4',
                            'Team1 position', 'Team2 position', 'Team3 position', 'Team4 position',
                            'Team1 result', 'Team2 result', 'Team3 result', 'Team4 result']


In [28]:
grouped_expanded

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,4th,1st
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,3rd,4th,1st
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,4th,1st,3rd
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,4th,3rd,2nd,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
96,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,eliminated,advancing,eliminated
97,8,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
98,9,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",München tja,WSDC Denmark Peter &amp; Haniya,INSEAD FT#3,Leiden Peter Singer enjoyers,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,eliminated,eliminated,eliminated


In [29]:
df6 = grouped_expanded
df6

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,4th,1st
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,3rd,4th,1st
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,4th,1st,3rd
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,4th,3rd,2nd,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
96,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,eliminated,advancing,eliminated
97,8,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
98,9,"Alex McCreadieⒸ , Amalie Bøgh-Sørensen , Heler...",München tja,WSDC Denmark Peter &amp; Haniya,INSEAD FT#3,Leiden Peter Singer enjoyers,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,eliminated,eliminated,eliminated


In [30]:
df6 = df6.drop(df6[df6['Team1'] == 'team'].index)
df6 = df6.reset_index()
df6 = df6.drop('index', axis=1)
df6

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,4th,1st
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,3rd,4th,1st
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,4th,1st,3rd
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,4th,3rd,2nd,1st
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,6,"Velina AndonovaⒸ , Leo Marinopoulos , Marília ...",Cambridge A,Belgrade BK,EUDU Wurdaan in Copenhagen? The west ha,Tartu A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,advancing,eliminated,eliminated
87,7,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",Oxford A,Ljubljana Hej Slovani,Tallinn unseen university a,Newydd Aburestvičevci A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,advancing,eliminated,eliminated
88,7,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",WSDC Denmark Coach Laura is abusive please help,LSE stands for Library Sex Enjoyers (a,Cambridge A,Belgrade BK,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,advancing,advancing,eliminated
89,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,eliminated,advancing,eliminated


In [31]:
df7 = df6
df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
for i in df7.index:
    loop_speaker1 = 'iron/swing'
    loop_speaker2 = 'iron/swing'
    for j in speakertab.index:
        if df7.Team1[i] == speakertab.team[j]:
            print(df7.Team1[i])
df7

Corvinus High taxes &amp; government deficit...
Corvinus High taxes &amp; government deficit...
Cambridge Copenhagen was a prep comp for Cop
Cambridge Copenhagen was a prep comp for Cop
München tja
München tja
SSE Riga Auh auh auuuu
SSE Riga Auh auh auuuu
WANTED: Witch Mommy
WANTED: Witch Mommy
Leiden Peter Singer enjoyers
Leiden Peter Singer enjoyers
Berlin ö to the ø
Berlin ö to the ø
Debate partners with drawbacks
Debate partners with drawbacks
München Munich🍺
München Munich🍺
Cambridge A
Cambridge A
LSE Take De Bait
LSE Take De Bait
Stockholm I&#x27;m Mid AF FML
Stockholm I&#x27;m Mid AF FML
Newydd Aburestvičevci A
Newydd Aburestvičevci A
Corvinus ... led to governmental collapse i
Corvinus ... led to governmental collapse i
our lovechild would look like jame
our lovechild would look like jame
Aachen Fishy fingers
Aachen Fishy fingers
Tallinn unseen university a
Tallinn unseen university a
Tartu Vi farede vild, hvor er Aarhus?
Tartu Vi farede vild, hvor er Aarhus?
Belgrade BK
Belgra

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result,Speaker1,Speaker2,Speaker1_score,Speaker2_score,Comp_name
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,4th,1st,0,0,0,0,Copenhagen2025
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,3rd,4th,1st,0,0,0,0,Copenhagen2025
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Opening Government,Opening Opposition,Closing Government,Closing Opposition,2nd,4th,1st,3rd,0,0,0,0,Copenhagen2025
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th,0,0,0,0,Copenhagen2025
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Opening Government,Opening Opposition,Closing Government,Closing Opposition,4th,3rd,2nd,1st,0,0,0,0,Copenhagen2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,6,"Velina AndonovaⒸ , Leo Marinopoulos , Marília ...",Cambridge A,Belgrade BK,EUDU Wurdaan in Copenhagen? The west ha,Tartu A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,advancing,eliminated,eliminated,0,0,0,0,Copenhagen2025
87,7,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",Oxford A,Ljubljana Hej Slovani,Tallinn unseen university a,Newydd Aburestvičevci A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,advancing,advancing,eliminated,eliminated,0,0,0,0,Copenhagen2025
88,7,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",WSDC Denmark Coach Laura is abusive please help,LSE stands for Library Sex Enjoyers (a,Cambridge A,Belgrade BK,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,advancing,advancing,eliminated,0,0,0,0,Copenhagen2025
89,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Opening Government,Opening Opposition,Closing Government,Closing Opposition,eliminated,eliminated,advancing,eliminated,0,0,0,0,Copenhagen2025


In [32]:
#speakertab.speaker[1]
speakertab 

,SpeakRank,speaker,categories,team,R1,R2,R3,R4,R5,Spk
0,1,Max Rosen,NaN,Cambridge A,83,82,83,84,83,415
1,2=,Gareth Lim Yefeng,NaN,Oxford A,79,83,84,85,81,412
2,2=,Bharath Anantham,NaN,Oxford A,79,83,82,86,82,412
3,4=,Aniket Godbole,NaN,LSE stands for Library Sex Enjoyers (a,84,79,81,87,79,410
4,4=,David Safro,ESL,Ljubljana Hej Slovani,83,77,85,84,81,410
...,...,...,...,...,...,...,...,...,...,...
136,137=,Speaker 1,NaN,Swing 2,—,—,—,—,71,71
137,137=,Speaker 2,NaN,Swing 2,—,—,—,—,71,71
138,139=,Speaker 1,NaN,Swing 3,—,—,—,—,—,—
139,139=,Speaker 2,NaN,Swing 3,—,—,—,—,—,—


## Part V. Formatting the data before saving

In [33]:
def get_chair(text):
    return text.split("Ⓒ")[0]

def get_wing1(paneltext):
    parts = paneltext.split(',')
    return parts[1] if len(parts) > 1 else None  # Returns second part if it exists

def get_wing2(paneltext):
    parts = paneltext.split(',')
    if len(parts) > 2:
        value = parts[2]
        return value if 'Ⓣ' not in value else 0
    return 0

def get_trainee1(paneltext):
    parts = paneltext.split(',')
    for part in parts:
        if 'Ⓣ' in part:
            return part
    return 0

def get_trainee2(paneltext):
    parts = [part for part in paneltext.split(',') if 'Ⓣ' in part]
    return parts[1] if len(parts) > 1 else 0

# Define column names
columns = ['Round', 'Panel', 
           'OG', 'OO', 'CG','CO',
           'OG speaker 1', 'OG speaker 2', 'OO speaker 1','OO speaker 2',
           'CG speaker 1', 'CG speaker 2', 'CO speaker 1','CO speaker 2',
           'OG1 score', 'OG2 score', 'OO1 score','OO2 score',
           'CG1 score', 'CG2 score', 'CO1 score','CO2 score',
           'Roundtype', 'Panel_size', 'Voting_panel_size',
           'Chair', 'Wing1', 'Wing2',
           'Trainee1', 'Trainee2', 'Comp_name']

# Get length of df9 
rownumbers = df9.shape[0] 

# Create an empty DataFrame with NaN (or pd.NA) values
data = pd.DataFrame(data={col: [0] * rownumbers for col in columns})

for i in df9.index:
    data.Round[i] = df9.Round[i]
    data.Panel[i] = df9.Panel[i]
    
    lineups = df9.iloc[i,6:10].tolist()
    pos_og = lineups.index('Opening Government')
    pos_oo = lineups.index('Opening Opposition')
    pos_cg = lineups.index('Closing Government')
    pos_co = lineups.index('Closing Opposition')
    teams_order = [pos_og, pos_oo, pos_cg, pos_co]
    
    data.OG[i] = df9.iat[i,teams_order[0]+2]
    data.OO[i] = df9.iat[i,teams_order[1]+2]
    data.CG[i] = df9.iat[i,teams_order[2]+2]
    data.CO[i] = df9.iat[i,teams_order[3]+2]
    
    data.iat[i,6] = df9.iat[i,teams_order[0]+14]
    data.iat[i,7] = df9.iat[i,teams_order[0]+18]
    data.iat[i,8] = df9.iat[i,teams_order[1]+14]
    data.iat[i,9] = df9.iat[i,teams_order[1]+18]
    data.iat[i,10] = df9.iat[i,teams_order[2]+14]
    data.iat[i,11] = df9.iat[i,teams_order[2]+18]
    data.iat[i,12] = df9.iat[i,teams_order[3]+14]
    data.iat[i,13] = df9.iat[i,teams_order[3]+18]
    
    if df9.iat[i,teams_order[0]+22] == 'outround':
        data.iat[i,14] = df9.iat[i,teams_order[0]+10]
        data.iat[i,15] = df9.iat[i,teams_order[0]+10]
        data.iat[i,16] = df9.iat[i,teams_order[1]+10]
        data.iat[i,17] = df9.iat[i,teams_order[1]+10]
        data.iat[i,18] = df9.iat[i,teams_order[2]+10]
        data.iat[i,19] = df9.iat[i,teams_order[2]+10]
        data.iat[i,20] = df9.iat[i,teams_order[3]+10]
        data.iat[i,21] = df9.iat[i,teams_order[3]+10]
        data.Roundtype[i] = "outround"
        
    else:
        data.iat[i,14] = df9.iat[i,teams_order[0]+22]
        data.iat[i,15] = df9.iat[i,teams_order[0]+26]
        data.iat[i,16] = df9.iat[i,teams_order[1]+22]
        data.iat[i,17] = df9.iat[i,teams_order[1]+26]
        data.iat[i,18] = df9.iat[i,teams_order[2]+22]
        data.iat[i,19] = df9.iat[i,teams_order[2]+26]
        data.iat[i,20] = df9.iat[i,teams_order[3]+22]
        data.iat[i,21] = df9.iat[i,teams_order[3]+26]
        data.Roundtype[i] = "inround"
        
    data.Panel_size[i] = df9.Panel[i].count(",")+1
    data.Voting_panel_size[i] = data.Panel_size[i] - df9.Panel[i].count("Ⓣ")
    if df9.Panel[i] != 0:
        data.Chair[i]  = get_chair(df9.Panel[i])
        
    if data.Panel_size[i] > 0:
        data.Wing1[i] = get_wing1(df9.Panel[i])
    if data.Panel_size[i] > 1:
        data.Wing2[i] = get_wing2(df9.Panel[i])
    if data.Panel_size[i] > 0:
        data.Trainee1[i] = get_trainee1(df9.Panel[i])
    if data.Panel_size[i] > 1:
        data.Trainee2[i] = get_trainee2(df9.Panel[i])
        
    data.Comp_name[i] = comp_name
data

C:\Users\andra\AppData\Local\Temp\ipykernel_32020\3157058536.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.OG[i] = df9.iat[i,teams_order[0]+2]
C:\Users\andra\AppData\Local\Temp\ipykernel_32020\3157058536.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.OO[i] = df9.iat[i,teams_order[1]+2]
C:\Users\andra\AppData\Local\Temp\ipykernel_32020\3157058536.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.CG[i] = df

,Round,Panel,OG,OO,CG,CO,OG speaker 1,OG speaker 2,OO speaker 1,OO speaker 2,...,CO2 score,Roundtype,Panel_size,Voting_panel_size,Chair,Wing1,Wing2,Trainee1,Trainee2,Comp_name
0,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Grace Graham,Botond Gergely Buday,Mirtel Etverk,Tommi Valgma,...,83,inround,2,1,Alex McCreadie,Naomi AdamⓉ,0,Naomi AdamⓉ,0,Copenhagen2025
1,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Fraser McConachie,Olivia Arbour,Louis Carl Aagaard Schou,Julius Juhl,...,81,inround,2,1,Ana Coman,Evangelia NtziokaⓉ,0,Evangelia NtziokaⓉ,0,Copenhagen2025
2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Philipp Steen,Adora Sabau,Hans Marten,Kaur Saarits,...,76,inround,2,1,Annie Schwartz,Dora GnadigⓉ,0,Dora GnadigⓉ,0,Copenhagen2025
3,1,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Lotte Tõnupärt,Sten Martin Kabrits,Daniël Hooimeijer,Katrina Cassar,...,73,inround,3,2,Heleriin Ots,Leonie Doerr,0,Jacob Rønde BerggreenⓉ,0,Copenhagen2025
4,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Anna Shkilenok,Stepan Sodel,Ulrich Wiebe,Adrian Buchwald,...,75,inround,3,1,Jack Stanley,Felipe PoviñaⓉ,0,Felipe PoviñaⓉ,Rasmus SchadeⓉ,Copenhagen2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,6,"Velina AndonovaⒸ , Leo Marinopoulos , Marília ...",Cambridge A,Belgrade BK,EUDU Wurdaan in Copenhagen? The west ha,Tartu A,Max Rosen,Sophie Hannigan,Obrad Kostic,Strahinja Batanjski,...,eliminated,outround,3,3,Velina Andonova,Leo Marinopoulos,Marília Montenegro,0,0,Copenhagen2025
87,7,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",Oxford A,Ljubljana Hej Slovani,Tallinn unseen university a,Newydd Aburestvičevci A,Gareth Lim Yefeng,Bharath Anantham,David Safro,Maj Hrovatin,...,eliminated,outround,5,5,Ana Coman,Andrew Mabon,Annie Schwartz,0,0,Copenhagen2025
88,7,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",WSDC Denmark Coach Laura is abusive please help,LSE stands for Library Sex Enjoyers (a,Cambridge A,Belgrade BK,Darin Mansurbeg,Maria Bahar Alamgiralam,Aniket Godbole,Stefan-Christian Milicescu,...,eliminated,outround,5,5,Jack Stanley,Lara Dragojević,Leo Marinopoulos,0,0,Copenhagen2025
89,8,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Aniket Godbole,Stefan-Christian Milicescu,Gareth Lim Yefeng,Bharath Anantham,...,eliminated,outround,5,5,Martha McKinney-Perry,Lara Dragojević,Maša Popesković,0,0,Copenhagen2025


In [34]:
reordered_columns = ['Comp_name', 'Round', 'Chair', 'Roundtype',
           'OG', 'OO', 'CG','CO',
           'OG speaker 1', 'OG speaker 2', 'OO speaker 1','OO speaker 2',
           'CG speaker 1', 'CG speaker 2', 'CO speaker 1','CO speaker 2',
           'OG1 score', 'OG2 score', 'OO1 score','OO2 score',
           'CG1 score', 'CG2 score', 'CO1 score','CO2 score',
           'Panel_size', 'Voting_panel_size', 'Panel',
           'Wing1', 'Wing2', 'Trainee1', 'Trainee2']
data = data[reordered_columns]
data

,Comp_name,Round,Chair,Roundtype,OG,OO,CG,CO,OG speaker 1,OG speaker 2,...,CG2 score,CO1 score,CO2 score,Panel_size,Voting_panel_size,Panel,Wing1,Wing2,Trainee1,Trainee2
0,Copenhagen2025,1,Alex McCreadie,inround,Corvinus High taxes &amp; government deficit...,baltic quota fillers,CDS The deliberators,LSE stands for Library Sex Enjoyers (a,Grace Graham,Botond Gergely Buday,...,73,84,83,2,1,"Alex McCreadieⒸ , Naomi AdamⓉ",Naomi AdamⓉ,0,Naomi AdamⓉ,0
1,Copenhagen2025,1,Ana Coman,inround,Cambridge Copenhagen was a prep comp for Cop,CDS The Debatinators,Berlin KaiserDAMN!,Separate but equal……… speeches,Fraser McConachie,Olivia Arbour,...,72,81,81,2,1,"Ana ComanⒸ , Evangelia NtziokaⓉ",Evangelia NtziokaⓉ,0,Evangelia NtziokaⓉ,0
2,Copenhagen2025,1,Annie Schwartz,inround,München tja,Tartu Kuulilennuteetunneliluuk,Tartu A,Oxford B,Philipp Steen,Adora Sabau,...,79,77,76,2,1,"Annie SchwartzⒸ , Dora GnadigⓉ",Dora GnadigⓉ,0,Dora GnadigⓉ,0
3,Copenhagen2025,1,Heleriin Ots,inround,SSE Riga Auh auh auuuu,Leiden Coping in the Hagen,2 Germans 1 Kebab,Imperial Su,Lotte Tõnupärt,Sten Martin Kabrits,...,76,75,73,3,2,"Heleriin OtsⒸ , Leonie Doerr , Jacob Rønde Ber...",Leonie Doerr,0,Jacob Rønde BerggreenⓉ,0
4,Copenhagen2025,1,Jack Stanley,inround,WANTED: Witch Mommy,WP Sanssouci Speakers,CDS Taking a 1st in socials,CDS Straight Outta Balkans,Anna Shkilenok,Stepan Sodel,...,76,78,75,3,1,"Jack StanleyⒸ , Felipe PoviñaⓉ , Rasmus SchadeⓉ",Felipe PoviñaⓉ,0,Felipe PoviñaⓉ,Rasmus SchadeⓉ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Copenhagen2025,6,Velina Andonova,outround,Cambridge A,Belgrade BK,EUDU Wurdaan in Copenhagen? The west ha,Tartu A,Max Rosen,Sophie Hannigan,...,eliminated,eliminated,eliminated,3,3,"Velina AndonovaⒸ , Leo Marinopoulos , Marília ...",Leo Marinopoulos,Marília Montenegro,0,0
87,Copenhagen2025,7,Ana Coman,outround,Oxford A,Ljubljana Hej Slovani,Tallinn unseen university a,Newydd Aburestvičevci A,Gareth Lim Yefeng,Bharath Anantham,...,eliminated,eliminated,eliminated,5,5,"Ana ComanⒸ , Andrew Mabon , Annie Schwartz , M...",Andrew Mabon,Annie Schwartz,0,0
88,Copenhagen2025,7,Jack Stanley,outround,WSDC Denmark Coach Laura is abusive please help,LSE stands for Library Sex Enjoyers (a,Cambridge A,Belgrade BK,Darin Mansurbeg,Maria Bahar Alamgiralam,...,advancing,eliminated,eliminated,5,5,"Jack StanleyⒸ , Lara Dragojević , Leo Marinopo...",Lara Dragojević,Leo Marinopoulos,0,0
89,Copenhagen2025,8,Martha McKinney-Perry,outround,LSE stands for Library Sex Enjoyers (a,Oxford A,Cambridge A,Ljubljana Hej Slovani,Aniket Godbole,Stefan-Christian Milicescu,...,advancing,eliminated,eliminated,5,5,"Martha McKinney-PerryⒸ , Lara Dragojević , Maš...",Lara Dragojević,Maša Popesković,0,0


In [35]:
teamtab.to_excel(comp_name+'_teamtab_done'+'.xlsx', index=True)
speakertab.to_excel(comp_name+'_speakertab_done'+'.xlsx', index=True)
data.to_excel(comp_name+'_rounds_done'+'.xlsx', index=True)